In [19]:
from itertools import chain, product
from sympy import simplify, sympify, Matrix, Dummy
from IPython.display import display
ichain = chain.from_iterable

X = lambda: Dummy('\\mathrm{X}')

subst = lambda vals: lambda expr: sympify(expr).subs(vals)

id = lambda x: x

valmap = lambda func, dict: dict if func is id else {k: func(v) for k, v in dict.items()}

prepend = lambda x, xs: chain((x,), xs)

def zip_dicts(first, *rest):
    keys = (*first.keys(),)
    def check_keys(dict):
        if set(keys) != set(dict.keys()):
            raise ValueError('Dictionaries must all share the same set of keys')
        return (dict[k] for k in keys)
    return keys, map(check_keys, chain((first,), rest))

truth_table = lambda n: product([False, True], repeat=n)

var_truth_table = lambda *vars: (dict(zip(vars, values)) for values in truth_table(len(vars)))

tabulated = lambda headers, *rows: Matrix([list(iter(row)) for row in prepend(headers, rows)])

merge_subs = lambda exprs, subs: subs | valmap(subst(subs), exprs)

self_subs = lambda eqs, n=1: self_subs(valmap(subst(eqs), eqs), n-1) if n > 0 else eqs

states = lambda symbs: ichain((symb, Symbol(f"{{{symb.name}}}'")) for symb in symbs)

simplify_ = simplify
bobby_tables = lambda eqs, vars, simplify=False: tabulated(
    *prepend(
        *zip_dicts(
            *(valmap(simplify_ if simplify else id, merge_subs(eqs, vals)
                ) for vals in var_truth_table(*vars))
            ))).subs({False: 0, True: 1})


In [20]:
from sympy import symbols, Tuple, true, false
q, qn = states(symbols('Q,'))
d, j, k = symbols("D J K")
jk_ff = lambda q, j, k, clk=true: (q & ~k | ~q & j) & clk | q & ~clk
d_ff = lambda q, d, clk=true: d & clk | q & ~clk

display(bobby_tables({qn: jk_ff(q, j, k)}, (j, k), simplify=True))
display(bobby_tables({qn: d_ff(q, d)}, (d,)))
jk_ff(q, j, true)

Matrix([
[J, K, {Q}'],
[0, 0,    Q],
[0, 1,    0],
[1, 0,    1],
[1, 1,   ~Q]])

Matrix([
[D, {Q}'],
[0,    0],
[1,    1]])

J & ~Q

In [21]:
from sympy import Eq, solve, true
bobby_tables({qn: jk_ff(q, j, k)}, (q, j, k), simplify=True)

# def solve_logic(expr, eq_to, deps, indeps):
#     for (vdeps, vindeps) in product(truth_table(len(deps), truth_table(len(indeps))):
#         substs = dict(ichain(zip(deps, vdeps), zip(indeps, vindeps)))
#         veq_to = expr.subs(substs)

# def solve_logic(expr, res, bound, free):
#     def reducer(accum, value):
#         result, free_values, bound_values = value
#         key = (result, *free_values)
#         if (same_result := accum.get(key, None)) is not None:
            
#         return accum | {(result, *free_values): }
        



Matrix([
[Q, J, K, {Q}'],
[0, 0, 0,    0],
[0, 0, 1,    0],
[0, 1, 0,    1],
[0, 1, 1,    1],
[1, 0, 0,    1],
[1, 0, 1,    0],
[1, 1, 0,    1],
[1, 1, 1,    0]])

In [22]:
from sympy import symbols

q1, q1n, q2, q2n = states(symbols('Q_1 Q_2'))
d1, j2, k2, a, z = symbols("D_1 J_2 K_2 A Z")

input_eqs = {d1: (~q1 | q2) & a, j2: k2, k2: ~a & q2 | a & q1 & ~q2}
output_eqs = {q1n: d_ff(q1, d1), q2n: jk_ff(q2, j2, k2), z: q1 & q2}

list(map(lambda x: dict(zip([a, q1, q2], x)), truth_table(3)))

bobby_tables(self_subs(input_eqs | output_eqs), (a, q1, q2))

Matrix([
[A, Q_1, Q_2, D_1, J_2, K_2, {Q_1}', {Q_2}', Z],
[0,   0,   0,   0,   0,   0,      0,      0, 0],
[0,   0,   1,   0,   1,   1,      0,      0, 0],
[0,   1,   0,   0,   0,   0,      0,      0, 0],
[0,   1,   1,   0,   1,   1,      0,      0, 1],
[1,   0,   0,   1,   0,   0,      1,      0, 0],
[1,   0,   1,   1,   0,   0,      1,      1, 0],
[1,   1,   0,   0,   1,   1,      0,      1, 0],
[1,   1,   1,   1,   0,   0,      1,      1, 1]])

In [23]:
from sympy import symbols, sympify, true, Eq, Matrix

ja, jb, jc, ka, kb, clkbn, kc = symbols("J_a J_b J_c K_a K_b {CLK_b}' K_c")
qa, qan, qb, qbn, qc, qcn = states(symbols('Q_a Q_b Q_c'))

input_eqs = {
    ka: true,
    ja: ~qc,
    kb: true,
    jb: true,
    kc: true,
    jc: qb & qa,
    clkbn: qa & ~qan
}

output_eqs = {
    qan: jk_ff(qa, ja, ka),
    qbn: jk_ff(qb, jb, kb, clk=clkbn),
    qcn: jk_ff(qc, jc, kc)
}

dict_to_eqs = lambda dict: (Eq(k, v) for k, v in dict.items())

display(Matrix([*dict_to_eqs(input_eqs | output_eqs)]))

bobby_tables(self_subs(input_eqs | output_eqs, n=2), (qa, qb, qc))

Matrix([
[                                                             Eq(K_a, True)],
[                                                             Eq(J_a, ~Q_c)],
[                                                             Eq(K_b, True)],
[                                                             Eq(J_b, True)],
[                                                             Eq(K_c, True)],
[                                                        Eq(J_c, Q_a & Q_b)],
[                                               Eq({CLK_b}', Q_a & ~{Q_a}')],
[                                   Eq({Q_a}', (J_a & ~Q_a) | (Q_a & ~K_a))],
[Eq({Q_b}', (Q_b & ~{CLK_b}') | ({CLK_b}' & ((J_b & ~Q_b) | (Q_b & ~K_b))))],
[                                   Eq({Q_c}', (J_c & ~Q_c) | (Q_c & ~K_c))]])

Matrix([
[Q_a, Q_b, Q_c, K_a, J_a, K_b, J_b, K_c, J_c, {CLK_b}', {Q_a}', {Q_b}', {Q_c}'],
[  0,   0,   0,   1,   1,   1,   1,   1,   0,        0,      1,      0,      0],
[  0,   0,   1,   1,   0,   1,   1,   1,   0,        0,      0,      0,      0],
[  0,   1,   0,   1,   1,   1,   1,   1,   0,        0,      1,      1,      0],
[  0,   1,   1,   1,   0,   1,   1,   1,   0,        0,      0,      1,      0],
[  1,   0,   0,   1,   1,   1,   1,   1,   0,        1,      0,      1,      0],
[  1,   0,   1,   1,   0,   1,   1,   1,   0,        1,      0,      1,      0],
[  1,   1,   0,   1,   1,   1,   1,   1,   1,        1,      0,      0,      1],
[  1,   1,   1,   1,   0,   1,   1,   1,   1,        1,      0,      0,      0]])

In [29]:
from sympy import symbols, Symbol
from sympy.logic import SOPform
from functools import reduce

states2 = lambda symbs: ((symb, Symbol(f"{{{symb.name}}}'")) for symb in symbs)

current_states = symbols("Q_1 Q_2 Q_3 Q_4")
*input_vars, = (symbols(f'J_{n} K_{n} D_{n}') for n in range(1, 5))
*state_pairs, = states2(current_states)
q1, q1n, q2, q2n, q3, q3n, q4, q4n = ichain(state_pairs)
invalid_states = [10, 11, 12, 13, 14, 15]
is_invalid = SOPform(current_states, invalid_states) & X()

nexts = {
    q1n: SOPform(current_states, [7, 8]) | is_invalid,
    q2n: SOPform(current_states, [3, 4, 5, 6]) | is_invalid,
    q3n: SOPform(current_states, [1, 2, 5, 6]) | is_invalid,
    q4n: SOPform(current_states, [0, 2, 4, 6, 8]) | is_invalid,
}


inputs = reduce(
    lambda a, b: a | b,
    ({j: j_inv(q, qn), k: k_inv(q, qn), d: d_inv(q, qn)
        } for (j, k, d), (q, qn) in zip(input_vars, state_pairs)),
)

display(Matrix([*dict_to_eqs(nexts)]))

bobby_tables(self_subs(nexts | inputs), current_states)

Matrix([
[                Eq({Q_1}', (Q_2 & Q_3 & Q_4 & ~Q_1) | (Q_1 & ~Q_2 & ~Q_3 & ~Q_4) | (_\mathrm{X} & ((Q_1 & Q_2) | (Q_1 & Q_3))))],
[Eq({Q_2}', (Q_2 & ~Q_1 & ~Q_3) | (Q_2 & ~Q_1 & ~Q_4) | (Q_3 & Q_4 & ~Q_1 & ~Q_2) | (_\mathrm{X} & ((Q_1 & Q_2) | (Q_1 & Q_3))))],
[                            Eq({Q_3}', (Q_3 & ~Q_1 & ~Q_4) | (Q_4 & ~Q_1 & ~Q_3) | (_\mathrm{X} & ((Q_1 & Q_2) | (Q_1 & Q_3))))],
[                                 Eq({Q_4}', (~Q_1 & ~Q_4) | (~Q_2 & ~Q_3 & ~Q_4) | (_\mathrm{X} & ((Q_1 & Q_2) | (Q_1 & Q_3))))]])

Matrix([
[Q_1, Q_2, Q_3, Q_4,      {Q_1}',      {Q_2}',      {Q_3}',      {Q_4}',         D_1,         D_2,         D_3,         D_4],
[  0,   0,   0,   0,           0,           0,           0,           1,           0,           0,           0,           1],
[  0,   0,   0,   1,           0,           0,           1,           0,           0,           0,           1,           0],
[  0,   0,   1,   0,           0,           0,           1,           1,           0,           0,           1,           1],
[  0,   0,   1,   1,           0,           1,           0,           0,           0,           1,           0,           0],
[  0,   1,   0,   0,           0,           1,           0,           1,           0,           1,           0,           1],
[  0,   1,   0,   1,           0,           1,           1,           0,           0,           1,           1,           0],
[  0,   1,   1,   0,           0,           1,           1,           1,           0,           1,           

In [27]:
from sympy.logic import bool_map
from sympy.abc import x, y ,z, a, b, c

q, qn = states(symbols('Q,'))
j, k = symbols("J K")

print(bool_map(x, y))

j_inv = lambda q, qn, x=X: q & x() | ~q & qn
k_inv = lambda q, qn, x=X: ~q & x() | q & ~qn
#def j_sop 
bobby_tables({j: j_inv(q, qn), k: k_inv(q, qn)}, (q, qn))
d_inv = lambda q, qn: qn


(x, {x: y})
